In [ ]:
import numpy as np 
import tensorflow as tf 

# Xây dựng phươngt thức tối ưu hóa qua trình đào tạo cho mô hình 
def create_train_op(loss, params): 
    # Lấy ra tham số lr của kháo lr từ , từ điển params 
    lr = params['lr']
    # kiểm tra xem giá trị warmup_steps trong từ điển params có = True không 
    # có nghĩa là mô hình co đang khởi động sẵn sàng hay không . 
    if "warmup_steps"  in params.key():
        # gán cho lrr bằng  giá trị của phuuwong thức cosine_decay_with warp 
        # phuuwong thức này cho phép phân dã tỷ lệ lr với mỗi bước warmup 
        r = cosine_decay_with_warmup(tf.train.get_global_step(), lr,
                                        params["max_steps"], warmup_steps=params["warmup_steps"])

    # kiểm tra xem trình tối ưu hóa của training có phải là adam 
    if params["opt_name"] == "adam":
        # kiểm tra xem weight_decay có trong từ điển params 
        if not "weight_decay" in params.keys():
            # Ta xây dựng trình tối ưu hóa adam 
            optimizer = tf.train.AdamOptimizer(
                lerning_rate = lr, 
                beta1 = params["beta1"], 
                beta2 = params["beta2"],
                epsilon=params["epsilon"]
            )
        
        # Trường hợp còn lại sử dụng AdamWOptimizer 
        else: 
            optimizer = tf.contrib.opt.AdamWOptimizer(
                learning_rate=lr,
                weight_decay=lr*params["weight_decay"],
                beta1=params["beta1"],
                beta2=params["beta2"],
                epsilon=params["epsilon"])
    # TRường hợp params["opt_name"] == "adafactor"
    elif params["opt_name"] == "adafactor":
        # kiểm tra khóa "decay_type" == adam
        if params["decay_type"] == "adam":
            # nếu có ta tính một tỷ lệ phân giã chuyền vào nó tham số beta2 của 
            # từ điển params 
            decay_rate = adafactor_decay_rate_adam(params["beta2"])
        #  nêu như = pow
        elif params["decay_type"] == "pow":
            # ta thực hiện như trên nhưng với tham số exponent 
            decay_rate = adafactor_decay_rate_pow(params["decay_exponent"])
        # trường hợp còn lại đưa ra cảnh báo 
        else:
            raise ValueError("unknown optimizer_adafactor_decay_type")
        # nếu weight_decay không có trong từ điển 
        if not "weight_decay" in params.keys():
            # sử dụng tối ưu hóa AdafactorOptimizer
            optimizer = AdafactorOptimizer(
                # Truyền vào các tham số lrr 
                learning_rate=lr,
                # tỷ lệ phân dã 
                decay_rate=decay_rate,
                # tham số beta
                beta1=params["beta1"],
                name="Adafactor")
        else:
            # đưa ra sử dụng một hàm từ Tensorflow để mửo rộng một lớp tối ưu hóa trong 
            # trường hợp này là AdafactorOptimizer. Hàm extend_with_decoupled_weight_decay
            # Thêm tính năng suy giảm trọng số không phụ thuộc vào lớp tối ưu hóa ban đầu 
            # điều này có nghĩa là có thể áp dụng giảm trọng sôs weight_decay mà không ảnh hưởng 
            # đến quá trình cập nhật Gradient trong trình huấn luyện
            AdafactorWOptimizer = tf.contrib.opt.extend_with_decoupled_weight_decay(AdafactorOptimizer)

            # User Optimizer AdafactorWOptimizer 
            optimizer = AdafactorWOptimizer(
                weight_decay=params["weight_decay"] * lr,
                learning_rate=lr,
                decay_rate=decay_rate,
                beta1=params["beta1"],
                name="AdafactorW")

    else:
        # trường hợp còn lại đưa ra 1 cảnh báo 
        raise ValueError("Unknown optimizer type!")

    # kiểm tra xem trình tối ưu có đang sử dụng TPU không 
    if params["use_tpu"]:
        # Phân bổ dữ liệu thông qua các phân mảnh TPU cho việc tối ưu hóa 
        optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

    # CẬP nhật trình tối ưu hóa cho mô hình 
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) # To update batchnorm, if present
    # Tối ưu hóa hàm loss cho trình tối ưuu
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    # sử dụng tf.group để nhóm các nhóm tham số train và update 
    train_op = tf.group([train_op, update_ops])

    # Trả về kết quả của train_op 
    return train_op



# Xây dựng phương thức cơ chức năng phân rã trọng số tỷ lệ học tập lr
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0,
                             name="learning_rate"):
    # kiểm tra xem tổng các bước có nhỏ hơn số bước khởi động hy không 
    if total_steps < warmup_steps: 
        # Đưa ra một cảnh báo 
        raise ValueError('total_steps must be larger or equal to '
                        'warmup_steps.')
    
    # Tính tỷ lệ học tập mới dựa trên cơ sở tỷ lệ học tập và số bước đã thực hiện 
    # Hàm tf.cos tạo ra một hàm giảm dần theo chu kỳ của hàm cosine giúp tỷ lệ học tập 
    # giảm một cachs mượt mà theo thời gian 
    learning_rate = 0.5 * learning_rate_base * (1 + tf.cos(
        np.pi *
        (tf.cast(global_step, tf.float32) - warmup_steps - hold_base_rate_steps
        ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    
    # nếu có bước giữ tỷ lệ cơ sỏ (hold_base_rate_steps) > 0
    # tywr lệ học tập sẽ được giữ nguyên là learning_rate_base cho đến khi số 
    # bước vượt qua ngưỡg này 
    if hold_base_rate_steps > 0:
        learning_rate = tf.where(
            # kiểm tra xem điều kiện đúng , nếu dúng dữ nguyên là lr, còn không 
            # tỷ lệ này sẽ được giữ nguyên là lrrb
            global_step > warmup_steps + hold_base_rate_steps, learning_rate, 
            learning_rate_base

        )
    # nếu như warmup_steps  > 0 :
    if warmup_steps > 0:
        # kiểm tra xem lr_base < warmup_lr không 
        if learning_rate_base < warmup_learning_rate:
            # nếu có đưa ra một cảnh báo 
            raise ValueError('learning_rate_base must be larger or equal to '
                        'warmup_learning_rate.')
        # sau đó tạo một biến slope 
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        # nhân slop với global_step + warmup_lr
        warmup_rate = slope * tf.cast(global_step,
                                    tf.float32) + warmup_learning_rate
        # kiểm tra  một điều kiện nếu đúng ta giữ nguyên giá trị của warmup_rate 
        # nếu không giữ nguyên giá trị lr
        learning_rate = tf.where(global_step < warmup_steps, warmup_rate,
                                learning_rate)
    # cuối cùng kiểm tra điều kiện và trả về kết quả tương ứng                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
    return tf.where(global_step > total_steps, 0.0, learning_rate,
                    name=name)



# Adafactor from tensor2tensor -------------------------------------------------------------

class AdafactorOptimizer(tf.train.Optimizer):
    """Optimizer that implements the Adafactor algorithm.
    Adafactor is described in https://arxiv.org/abs/1804.04235.
    Adafactor is most similar to Adam (Kingma and Ba), the major differences are:
    1. For a two-dimensional AxB weight matrix, Adafactor uses only A+B auxiliary
        parameters to maintain the second-moment estimator, instead of AB.
        This is advantageous on memory-limited systems.  In addition, beta1
        (momentum) is set to zero by default, saving an additional auxiliary
        parameter per weight.  Variables with >=3 dimensions are treated as
        collections of two-dimensional matrices - factorization is over the final
        two dimensions.
    2. Adafactor incorporates "update-clipping" - a scale-invariant analog of
        gradient clipping.  This adds stability
    3. Adafactor does not require an external "learning rate".  By default, it
        incorporates a relative-update-scale schedule, corresponding to
        inverse-square-root learning-rate-decay in ADAM.  We hope this works well
        for most applications.
    ALGORITHM:
    parameter -= absolute_update_scale * clip(grad / grad_scale)
    where:
        absolute_update_scale := relative_update_scale * parameter_scale
        relative_update_scale := min((step_num + 1)**-0.5, 1e-2)
        parameter_scale := max(rms(var)), epsilon2)
        clip(x) := x / max(1.0, rms(x))
        grad_scale := tf.sqrt(v)   (v is the second-moment estimator)
    The second-moment estimator v is maintained in a manner similar to Adam:
    We initialize
    ```
    if var is 2-dimensional:
        v_r <- zeros([num_rows])
        v_c <- zeros([num_cols])
    if var is 0-dimensional or 1-dimensional:
        v <- zeros(shape(var))
    ```
    The update rule is as follows:
    ```
    decay_rate = 1 - (step_num + 1) ^ -0.8
    grad_squared = tf.square(grad) + epsilon1
    if var is 2-dimensional:
        v_r <- decay_rate * v_r + (1 - decay_rate) * reduce_mean(grad_squared, 1)
        v_c <- decay_rate * v_c + (1 - decay_rate) * reduce_mean(grad_squared, 0)
        v = outer_prod(v_r, v_c) / reduce_mean(v_r)
    if var is 0-dimensional or 1-dimensional:
        v <- decay_rate * v + (1 - decay_rate) * grad_squared
    ```
    For variables with >=3 dimensions, we factorize the second-moment accumulator
    over the final 2 dimensions.  See the code for details.
    Several parts of this algorithm are configurable from the initializer.
        multiply_by_parameter_scale:  If True, then compute absolute_update_scale
        as described above.  If False, let absolute_update_scale be the externally
        supplied learning_rate.
        learning_rate: represents relative_update_scale if
        multiply_by_parameter_scale==True, or absolute_update_scale if
        multiply_by_parameter_scale==False.
        decay_rate: Decay rate of the second moment estimator (varies by step_num).
        This should be set to a function such that:
        1-1/(step_num + 1) <= decay_rate(step_num) < 1.0
        beta1: enables momentum, as in Adam.  Uses extra memory if nonzero.
        clipping_threshold: should be >=1.0 or None for no update clipping
        factored: whether to factor the second-moment estimator.  True means
        less memory usage.
    """

    def __init__(self,
                multiply_by_parameter_scale=True,
                learning_rate=None,
                decay_rate=None,
                beta1=0.0,
                clipping_threshold=1.0,
                factored=True,
                use_locking=False,
                name="Adafactor",
                epsilon1=1e-30,
                epsilon2=1e-3):
        """Construct a new Adafactor optimizer.
        See class comment.
        Args:
        multiply_by_parameter_scale: a boolean
        learning_rate: an optional Scalar.
        decay_rate: an optional Scalar.
        beta1: a float value between 0 and 1
        clipping_threshold: an optional float >= 1
        factored: a boolean - whether to use factored second-moment estimator
            for 2d variables
        use_locking: If True use locks for update operations.
        name: Optional name for the operations created when applying gradients.
            Defaults to "AdafactorOptimizer".
        epsilon1: Regularization constant for squared gradient.
        epsilon2: Regularization constant for parameter scale.
        Raises:
        ValueError: if absolute_update_scale and relative_update_scale_fn are both
            present or both absent.
        """
        super(AdafactorOptimizer, self).__init__(use_locking, name)
        self._multiply_by_parameter_scale = multiply_by_parameter_scale
        if learning_rate is None:
            learning_rate = self._learning_rate_default(multiply_by_parameter_scale)
        self._learning_rate = learning_rate
        if decay_rate is None:
            decay_rate = self._decay_rate_default()
        self._decay_rate = decay_rate
        self._beta1 = beta1
        self._clipping_threshold = clipping_threshold
        self._factored = factored
        self._epsilon1 = epsilon1
        self._epsilon2 = epsilon2

    # Thiết lập phương thức sử dụng công cụ ước tính thời đuểm thứ 2 
    # dựa trên hình dnagj của biến và trả về 1 danh sách kiểu boolean 
    def _should_use_factored_second_moment_estimate(self, shape):
        """Should we use a factored second moment estimator.
            Based on the shape of the variable.

        ARGS: 
            shape: A list of intergers
            Return: A boolean 
        """
        
        return self._factored and len(shape) >= 2 
    
    # thiết lập phương thức create_slot để tạo các "slot" cho các biến 
    # các slot này được sử dụng để lưu trữ thông tin cần thiết cho quá trình tối ưu hóa 
    def _create_slots(self, var_list):
        # duyệt qua các phần tử trong danh sách varlist 
        for var in var_list:
            # lấy ra hình dạng của var các hình dnagj này được trả về dưới dnagj list 
            shape = var.get_shape().as_list()
            # kiểm tra xem có sử dụng moment đầu tiên không 
            if self._beta1:
                # tao slot cho moment đầu tiên nếu beta_1 được thiết lập 
                self._zeros_slot(var, 'm', self._name)
            
            # kiểm tra điều kiện nếu kích thước của gradient lớn hơn 2 và sử dụng ước lượng moment
            # thứ 2 theo từng phần 
            if self._should_use_factored_second_moment_estimate(shape):
                # tạo một tensor shape là kích thước theo hàng ngang 
                # type = tf.float32
                r_var = tf.zeros(shape[:-1], dtype= tf.float32)
                # và tạo một ma trận theo cột 
                c_var = tf.zeros(shape[:-2] + shape[-1:], dtype=tf.float32)
                # Tạo slot cho ước lượng hàng 
                self._get_or_make_slot(var, r_var, "vr", self._name)
                # tạo một slot cho ước lượng cột 
                self.get_or_make_slot(var, c_var, "vc", self._name)
            
            else: 
                # Tạo mảng zeros cho full tensor 
                v_var = tf.zeros(shape, dtype=tf.float32)
                # Tạo slot cho moment thứ 2 
                self._get_or_make_slot(
                    var, v_var, "v", self._name
                )

    # Xây dựng các phuuwong thức để áp dụng các hàm chức năng cho việc xử lý một cách thu gọn 
    def _apply_dense(self, grad, var):
        # áp dụng phuuwong thức resource_apply_dense cho garadient và giá trị var 
        return self._resource_apply_dense(grad, var)
    
    def _apply_sparse(self, grad, var):
        return self._apply_dense(tf.convert_to_tensor(grad), var)

    def _resource_apply_sparse(self, grad, handle, indices):
        return self._resource_apply_dense(
            tf.convert_to_tensor(tf.IndexedSlices(grad, indices, tf.shape(handle))),
            handle)
    
    # Thiết lập phương thức thay đổi các tỷ lệ của tham số 
    # nhận đầu vào là biến var là một thực thể của tf.Variable . là một loại 
    # biến  đặc biệt được sử dụng trong tensorflow được sử dụng trong tensorflow 
    # để lưu trữ và cập nhật các trạng thái khi chạy mô hình 
    # là một tensor mà giá trị có thể thay đổi qua các phép biến đổi 
    def _parameter_scale(self, var):
        """Estimate the scale of the parameters from the current values.
            We include a minimum value of 0.001 to give it a chance to escape 0
        if it was zero-initialized.
        Instead of using the value, we could impute the scale from the shape,
        as initializers do.
        Args:
        var: a variable or Tensor.
        Returns:
        a Scalar
        """
        return tf.maximum(reduce_rms(var), self._epsilon2)
    
    # Thiết lập phương thức resource_apply_dense để áp dụng các lớp xử lý lên 
    # gard 9laf grad của mất mát , và handle tham chiếu đến biến có thể huấn luyện mà 
    # bạn muốn cập nhật . Trong tensorflow các biến thường được quản lý thông qua các 
    # handle để tối ưu hóa hiệu suất 
    def _resource_apply_dense(self, grad, handle):
        # gán cho var = handle 
        var = handle 
        # thay đổi kiểu datatype cho tham só grad 
        grad = tf.to_float(grad)
        # tính bình phương của giá trị grad sau đó cộng tham ố e
        grad_squared = tf.square(grad) + self._epsilon1
        # Tính trung bình giá trị của bộ tham số này 
        rad_squared_mean = tf.reduce_mean(grad_squared)
        # Định nghĩa tham số decay_rate , lr , var (tensor lưu trữ)
        decay_rate = self._decay_rate
        update_scale = self._learning_rate
        old_val = var

        # kiểm tra xem kểu dtype cơ sở của tensor var = bfloat16
        if var.dtype.base_dtype == tf.float16:
            # nếu đúng tensor này sẽ được chuyển qua kiểu float 
            # sử dụng self._parameter_encoding.decode để né / lượng tử hóa tham số pld_val 
            # Giải nén . chuyển đổi bfloat16 nén sang định dạng float32 đầy đủ 
            # Lượng tử hóa : Áp dụng các phép toán đảo ngược để khôi phục giá trị bfloat32 ban đầu 
            old_val = tf.to_float(self._parameter_encoding.decode(old_val))

        # nếu có sử dụng tỷ lệ tham số 
        if self._multiply_by_parameter_scale:
            # Nhân tỷ lệ này với tỷ lệ tham số chuyển đổi qua float32 
            update_scale *= tf.to_float(self._parameter_scale(old_val))    
        # HACK : tạo một sự phụ thuộc vào gard , điều này làm rối trí trình biên dịch XLA 
        # và ngăn chặn việc hợp nhất các tính toán giữa các biến khác nhau . Việc hợp nhất 
        # này không tốt cho việc sử dụng HBM vì nó khiên cho gradient tồn tại trong bộ nhớ 
        # lâu hơn cần thiết 
        ecay_rate += grad_squared_mean * 1e-30
        update_scale += grad_squared_mean * 1e-30
        # END HACK
        mixing_rate = 1.0 - decay_rate
        # Lấy ra hình dạng của tensor lưu trữ lưu dưới dạng list
        shape = var.get_shape().as_list()
        # tạo một danh sách để lưu trữ các giá trị dã được cập nhật 
        updates = []
        # kiểm tra xem có sử dụng ước lượng moment bậc 2 đã đựo phân tích hay không 
        # dựa trên hình dạng của tensorflow
        if self._should_use_factored_second_moment_estimate(shape):
            # tính trung bình bình phương gradient theo hàng avf cột 
            grad_squared_row_mean = tf.reduce_mean(grad_squared, -1)
            grad_squared_col_mean = tf.reduce_mean(grad_squared, -2)
            # sau đó lấy ácc slot vr và vc liên quan đến biến var 
            vr = self.get_slot(var, "vr")
            # Tính toán giá trị mới cho vr và vc dựa trên tỷ lệ suy giảm decay , tỷ lệ trộn
            # Và trung bình , bình phương  gradient 
            new_vr = (decay_rate * vr + mixing_rate * grad_squared_row_mean)
            vc = self.get_slot(var, "vc")
            new_vc = (decay_rate * vc + mixing_rate * grad_squared_col_mean)
            # Cập nhật giá trị cho vr và vc
            # sử dụng hàm assign(chỉ định) để cập nhật giá trị của 1 biến với 1 giá trị mới 
            # khi gọi assigh nó tạo ra một phép toán (operator) mà cần chạy một cách ro ràng để 
            # cập nhật các biến đó 
            vr_update = tf.assign(vr, new_vr, use_locking=self._use_locking)
            vc_update = tf.assign(vc, new_vc, use_locking=self._use_locking)
            updates = [vr_update, vc_update]
            # Tính trung bình dài hạn của new_vr 
            long_term_mean = tf.reduce_mean(new_vr, -1, keepdims=True)
            # # Tính nghịc đảo của căn bâc 2 của tỷ lệ giữa new_vr và long_term_mean 
            # và nghịc đảo của căn bậc 2new_vc 
            r_factor = tf.rsqrt(new_vr / long_term_mean)
            c_factor = tf.rsqrt(new_vc)
            # nhân gradient với các yếu tố r_factor và c_factor sau khi mở rộng chiều
            x = grad * tf.expand_dims(r_factor, -1) * tf.expand_dims(c_factor, -2)
        else:
            # trường hợp còn lại 
            # lấy các slot vr trong biến v
            v = self.get_slot(var, "v")
            # nhân các slot này với tỷ lệ phân dã decay , tỷ lệ trộn , bình phưuowng grad
            # để tạo thành v mới 
            new_v = decay_rate * v + mixing_rate * grad_squared
            # sau đó cập nhật v mới cho v 
            v_update = tf.assign(v, new_v, use_locking=self._use_locking)
            # Thêm v đã được vào danh sách chứa các giá trị đã cập nhật 
            updates = [v_update]
            # nhânn grad vơis căn bậc 2 của v mới 
            x = grad * tf.rsqrt(new_v)
        # kiểm tra xem ngưỡng cắt xém = None 
        if self._clipping_threshold is not None:
            # nếu không tính toán một giá trị 
            clipping_denom = tf.maximum(1.0, reduce_rms(x) / self._clipping_threshold)
            # chia x cho giá trị đã được tiunhs toán 
            x /= clipping_denom

        # nhân x với tỷ lệ update_scale 
        subtrahend = update_scale * x
        # kiểm tra xem beta1: a float value between 0 and 1
        if self._beta1: 
            # lấy các slot của m trong var 
            m = self.get_slot(var, "m")
            # tạo một gía trị cho m mới 
            new_m = self._beta1 * tf.to_float(m) + (1.0 - self._beta1) * subtrahend
            # gán cho subtrahend = new_m
            subtrahend = new_m
            # chuyển đổi new_m thành 1 tensor dtype = dtype.var
            new_m = cast_like(new_m, var)
            # Cập nhật m bằng m mới và thêm vào danh sacchs update 
            updates.append(tf.assign(m, new_m, use_locking=self._use_locking))
        # Thay đổi old val thành kiểu float sau đó trừ subtrahend rồi gán kết quả cho new_val 
        new_val = tf.to_float(old_val) - subtrahend
        # Cập nhật var mới 
        var_update = tf.assign(var, new_val, use_locking=self._use_locking)
        # Thêm giá trị đó vào danh sách update 
        updates = [var_update] + updates
        # Cuối cùng nhóm các tensor var trong update lại với nhau 
        return tf.group(*updates)

    # Xây dựng phương thức định nghĩa độ phân giã mặc định 
    def _decay_rate_default(self):
        return adafactor_decay_rate_pow(0.8)

    # Tương tự vơis learning_rate
    def _learning_rate_default(self, multiply_by_parameter_scale):
        # gán giá trị cho lr
        learning_rate = tf.minimum(tf.rsqrt(step_num() + 1.0), 0.01)
        # kiểm tra xem có tồn tại một đa sử lý quy mô tham số không 
        if not multiply_by_parameter_scale:
            # nêud có nhân tỷ lẹ lr với 0.05 
            learning_rate *= 0.05
        return learning_rate


# tỷ lệ phân giã cho trình tối ưu hóa adam 
def adafactor_decay_rate_adam(beta2):
    t = tf.to_float(tf.train.get_or_create_global_step()) + 1.0
    decay = beta2 * (1.0 - tf.pow(beta2, t - 1.0)) / (1.0 - tf.pow(beta2, t))
    # decay = tf.cond(tf.equal(t, 1.0), lambda: beta2, lambda: decay)
    return decay

# tỷ lệ lũy thừ tham số cho decay 
def adafactor_decay_rate_pow(exponent):
    return 1.0 - tf.pow((step_num() + 1.0), -exponent)

# tính toán  tổng các bước 
def step_num():
    return tf.to_float(tf.train.get_or_create_global_step())

# Tính mean_squared 
def reduce_rms(x):
    return tf.sqrt(tf.reduce_mean(tf.square(x)))

# Chuyển đổi tensor 
def cast_like(x, y):
    """Cast x to y's dtype, if necessary."""
    x = tf.convert_to_tensor(x)
    y = tf.convert_to_tensor(y)

    if x.dtype.base_dtype == y.dtype.base_dtype:
        return x

    cast_x = tf.cast(x, y.dtype)
    if cast_x.device != x.device:
        x_name = "(eager Tensor)"
        try:
            x_name = x.name
        except AttributeError:
            pass
        tf.logging.warning("Cast for %s may induce copy from '%s' to '%s'", x_name,
                        x.device, cast_x.device)
    return cast_x

In [2]:
import tensorflow as tf 

In [3]:
shape = [4, 3, 2]  # Kích thước ban đầu của tensor
r_val = tf.zeros(shape[:-1], dtype=tf.float32)  # Sẽ tạo một tensor mới với kích thước [4, 3]
c_val = tf.zeros(shape[:-2] + shape[-1:], dtype=tf.float32)  # Sẽ tạo một tensor mới với kích thước [4, 2]


In [5]:
r_val

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [6]:
e_val = tf.zeros(shape[:-2], dtype=tf.float32)
e_val

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>